In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sqlite3

In [ ]:
Veri_Seti3_201701_In = pd.read_csv('./Dataset 3/Dataset 3_201701_In.csv')

In [ ]:
pd.DataFrame(Veri_Seti3_201701_In, index = [0,1,2,3,4])

## Extract information from January dataset

In [ ]:
JanuaryIn = np.asarray(Veri_Seti3_201701_In)

In [ ]:
length, width = JanuaryIn.shape

In [ ]:
print ('there are {} calls recorded in this dataset'.format(length))

### Extract timestamp

In [ ]:
timestamp = pd.to_datetime(JanuaryIn[0:length,1], format="%d-%m-%Y %H:%M")

In [ ]:
date = np.asarray(timestamp.date)

### Extract called ID

In [ ]:
callerID = JanuaryIn[:,0]

In [ ]:
callerID = callerID.astype(str)

### Extract prefix

In [ ]:
prefix = []
for i in callerID:
    prefix.append(i[0])

prefix = np.asarray(prefix)
prefix = prefix.astype(int)

In [ ]:
unique, counts = np.unique(prefix, return_counts=True)
print ('there are {} refugee calls in this dataset'.format(counts[1]))

### Extract city ID

In [ ]:
cityID = JanuaryIn[:,3]

## Analyse data

In [ ]:
to_analyse = np.unique(callerID[prefix==2])

Accessing date

In [ ]:
h1 = callerID[timestamp.hour >= 18]
h2 = callerID[timestamp.hour <= 7]
h = np.append(h1,h2)
overlap = np.intersect1d(h,to_analyse)

In [ ]:
evening_location = {}
for i in overlap:
    evening_location[str(i)] = {
        'cities':[],
        'dates':[]
    }

In [ ]:
#bottleneck - fix this
bar = progressbar.ProgressBar(maxval=len(overlap), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

counter = 0
for i in overlap:
    location =  np.where(callerID == i)
    cities = cityID[location[0]]
    evening_location[i]['cities'] = cities
    dates = date[location[0]]
    evening_location[i]['dates'] = dates
    counter += 1
    #if counter%100 == 0:
    #    print (counter)
    bar.update(counter+1)
#    sleep(0.1)
bar.finish()

Save evening_locations file

In [ ]:
with open("evening_location.pkl", "wb") as fp:   #Pickling
    pickle.dump(evening_location, fp)

In [ ]:
with open("evening_location.pkl", "rb") as fp:   # Unpickling
    evening_location = pickle.load(fp)

In [ ]:
mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in evening_location:
    unique, counts = np.unique(evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        mobile.append(i)

In [ ]:
print ('there are {} callers out ot 50,000 refugees who moved cities'.format(len(mobile)))

Calculated cities moved between (i.e. were in 'residence' for for more that 4 nights) 

In [ ]:
#for i in mobile:
#    moved = [evening_location[i]['cities'][0]]
#    date_moved =  date[0]
#    for j in range(1,len(evening_location[i]['cities'])):
#        if evening_location[i]['cities'][j] != evening_location[i]['cities'][j-1]:
#            diff = np.timedelta64(evening_location[i]['dates'][j-1]-date_moved)
#            diff = int(diff.astype('timedelta64[D]') / np.timedelta64(1, 'D'))
#            if diff >=4:
#                moved.append(evening_location[i]['cities'][j-1])
#        date_moved = evening_location[i]['dates'][j]
#    evening_location[i]['moved'] = moved

In [ ]:
for i in mobile:
    moved = []
    moved.append(evening_location[i]['cities'][0])
    counter = 0
    for j in range(1,len(evening_location[i]['cities'])):
        if evening_location[i]['cities'][j] == evening_location[i]['cities'][j-1]:
            counter += 1
        else:
            counter = 0
        if counter == 4 and evening_location[i]['cities'][j] != moved[-1]:
            moved.append(evening_location[i]['cities'][j])
    evening_location[i]['moved'] = moved

In [ ]:
low_mobility = []
medium_mobility = []
high_mobility = []
for i in mobile:
    if len(evening_location[i]['moved']) <=2:
        low_mobility.append(i)
    elif 2 < len(evening_location[i]['moved']) <=5:
        medium_mobility.append(i)
    else:
        high_mobility.append(i)

In [ ]:
print ('There are {} refugees classed as low mobility'.format(len(low_mobility)))
print ('There are {} refugees classed as medium mobility'.format(len(medium_mobility)))
print ('There are {} refugees classed as high mobility'.format(len(high_mobility)))

### Compare against non-refugee population

In [ ]:
non_to_analyse = np.unique(callerID[prefix==1])

In [ ]:
non_h1 = callerID[timestamp.hour >= 18]
non_h2 = callerID[timestamp.hour <= 7]
non_h = np.append(non_h1,non_h2)
non_overlap = np.intersect1d(non_h,non_to_analyse)

In [ ]:
non_evening_location = {}
for i in non_overlap:
    non_evening_location[str(i)] = {
        'cities':[],
        'dates':[]
    }

In [ ]:
##bottleneck - could parallelise better
for i in non_overlap:
    location =  np.where(callerID == i)
    cities = cityID[location[0]]
    non_evening_location[i]['cities'] = cities
    dates = date[location[0]]
    non_evening_location[i]['dates'] = dates

In [ ]:
with open("non_evening_location.txt", "wb") as fp:   #Pickling
    pickle.dump(non_evening_location, fp)

In [ ]:
with open("non_evening_location.txt", "rb") as fp:   # Unpickling
    non_evening_location = pickle.load(fp)

In [ ]:
non_mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in non_evening_location:
    unique, counts = np.unique(non_evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        non_mobile.append(i)

In [ ]:
print ('there are {} callers out ot 50,000 non-refugees who moved cities'.format(len(non_mobile)))

In [ ]:
print ('there are {} times as many refugees who moved cities as non-refugees'.format(
    round(float(len(mobile))/float(len(non_mobile)),2)))